In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset, load_from_disk
from huggingface_hub import login

from utils.helpers import find_root_dir

from access_tokens import HF_WRITE_TOKEN

In [ ]:
login(token=HF_WRITE_TOKEN)
ARDT_DIR = find_root_dir()
MIN_EP_LEN = 20

In [ ]:
dataset = load_from_disk(f'{ARDT_DIR}/datasets-raw/nonadv-raw-data/ppo_hopper_v1_test_v1')

In [ ]:
pr_actions = dataset['actions']
adv_actions = [list(np.array(a) * 0.0) for a in pr_actions if len(a) > MIN_EP_LEN]

dataset = dataset.add_column('pr_actions', pr_actions)
dataset = dataset.add_column('adv_actions', adv_actions)
dataset = dataset.remove_columns(['actions'])

In [ ]:
total_number_of_steps = 0
for l in dataset['pr_actions']:
    total_number_of_steps += len(l)

print(total_number_of_steps)

In [ ]:
def compute_sum(ds):
    return {'returns': sum(ds['rewards'])}

ds_to_vis = dataset.map(compute_sum)
sns.displot(ds_to_vis['returns'], kind="kde", bw_adjust=0.5);

In [ ]:
# TYPE = 'train'
# VERSION = 'v4'

# dataset.save_to_disk(f'{ARDT_DIR}/datasets/ppo_noadv_{TYPE}_halfcheetah_{VERSION}')